In [1]:
import numpy as np
import pandas as pd

from nupic.frameworks.opf.model_factory import ModelFactory
from nupic.algorithms import anomaly_likelihood
import model_params

#IMPORT FOR PLOT
import plotly
import plotly.plotly as py
import plotly.graph_objs as go 
plotly.offline.init_notebook_mode(connected=True)

import datetime
from datetime import timedelta
DATE_FORMAT = "%m/%d/%y %H:%M:%S"


In [2]:
#Load of dataset
frame=pd.read_table('CalIt2.data', header=None, delimiter=",")
#Load of dataset events
frameEV=pd.read_table('CalIt2.events', header=None, delimiter=",")
frameEV=frameEV.drop(5)#remove event 5(it's at the same time of another)

In [3]:
#Function to create the model
def createModel(params):
  return ModelFactory.create(params)

In [19]:
#It creates the grey shapes of the events to show them on the chart
shapes=[]
for i in range(0, frameEV.shape[0]):
    shapes.append({
            'type': 'rect',
            'xref': 'x',
            'yref': 'paper',
            'x0': datetime.datetime.strptime(frameEV.iloc[i,0]+' '+frameEV.iloc[i,1], DATE_FORMAT),
            'y0': 0,
            'x1': datetime.datetime.strptime(frameEV.iloc[i,0]+' '+frameEV.iloc[i,2], DATE_FORMAT),
            'y1': 1,
            'fillcolor': '#000000',
            'opacity': 0.2,
            'line': {
                'width': 0,
            }
        })

In [5]:
#Extraction of the data of in and out from the frame
in__=frame.loc[frame[0] == 9]
out__=frame.loc[frame[0] == 7]

In [6]:
#It run a HTM model(if is None a new one is created)
def runHTM(params, inputData, timeData, anomalyLikelihood, model=None):
    if(model==None):
        model = createModel(params)
        model.enableInference({'predictedField': 'valuee'})
    
    timestamps=[]
    anomalyScore=[]
    likelihood=[]

    for i in range(0, inputData.shape[0]):
        timestamp = datetime.datetime.strptime(timeData.iloc[i,0]+' '+timeData.iloc[i,1], DATE_FORMAT)
        val = float(inputData[i,0])

        result = model.run({
          "timestamp": timestamp,
          "valuee": val
        })
        aS = result.inferences['anomalyScore']
        
        ld = anomalyLikelihood.anomalyProbability(
            {
              "timestamp": timestamp,
              "valuee": val
            }, aS#, timestamp
        )
        ld = anomalyLikelihood.computeLogLikelihood(ld)
        
        timestamps.append(timestamp)
        anomalyScore.append(aS)
        likelihood.append(ld)    
    return timestamps, anomalyScore, likelihood, model

In [47]:
#Creation of the charts with data and shapes(shap parameter)
#s is for start and e for end
def plotData(data, shap, message='Data vs AS vs AL', shapes2=[], s=None, e=None):
    print(message)
    dataG= []
    
    for key, value in data.items():
        if(key!="timestamps"):
            if(type(value).__module__ == np.__name__ and len(value.shape)==2):
                value=value[:,0]
            op=1.0
            if("anomalyScore" in key):
                op=0.3
            if("anomaly" in key):
                dataG.append(go.Scatter(y=value[s:e],
                                x=data["timestamps"][s:e],
                                yaxis='y2',
                                opacity=op,
                                name=key))
            else:
                dataG.append(go.Scatter(y=value[s:e],
                                x=data["timestamps"][s:e],
                                name=key))    
    layout = dict(
        title=message,        
        width=1000,
        height=450,
        xaxis=dict(
            rangeselector=dict(),
            rangeslider=dict()
        ),
        yaxis=dict(
            title='persone'
        ),
        yaxis2=dict(
            title='percentuale',
            overlaying='y',
            side='right'
        ),
        shapes=shap+shapes2
    )
    fig = dict(data=dataG, layout=layout)
    plotly.offline.iplot(fig)    

In [30]:
#Evaluation of the result by calculation of anomalies and characterization of them 
#as right prediction and bad prediction
#
#It print the result and return the shapes(for the chart) of the events right predicted 
def evaluateResult(threshold, inputL,timestamps, eventData):
    startT=None
    endT=None
    Nalarm=0
    NTrueAlarm=0
    used=[]
    shapes2=[]
    
    for j in range(0,len(inputL)):
        if(inputL[j]>=threshold and startT==None):
            startT=timestamps[j]
            Nalarm=Nalarm+1
            
        if(inputL[j]<threshold and startT!=None):
            endT=timestamps[j]
            flag=True
            for k in range(0,frameEV.shape[0]):
                s=datetime.datetime.strptime(eventData.iloc[k,0]+' '+eventData.iloc[k,1], DATE_FORMAT)
                e=datetime.datetime.strptime(eventData.iloc[k,0]+' '+eventData.iloc[k,2], DATE_FORMAT)

                if(startT<=e+timedelta(minutes=60) and endT>=s-timedelta(minutes=35)):
                    if(k in used):
                        if(flag):
                           Nalarm=Nalarm-1# if here means alarm counted but it's right so don't count 2 times
                           flag=False #to decrement Nalarm only one time because this prediction is already used
                                      #so it will be already considered
                    else:                   
                        NTrueAlarm+=1
                        used.append(k)
                        shapes2.append({
                            'type': 'rect',
                            'xref': 'x',
                            'yref': 'paper',
                            'x0': datetime.datetime.strptime(frameEV.iloc[k,0]+' '+frameEV.iloc[k,1], DATE_FORMAT),
                            'y0': 0,
                            'x1': datetime.datetime.strptime(frameEV.iloc[k,0]+' '+frameEV.iloc[k,2], DATE_FORMAT),
                            'y1': 1,
                            'fillcolor': '#ff0000',
                            'opacity': 0.2,
                            'line': {
                                'width': 0,
                            }
                        })
            startT=None
            endT=None
    print("Anomalie totali:" + str(Nalarm))
    print("Anomalie reali:" + str(NTrueAlarm))
    print("Percentuale prese:" + str((NTrueAlarm/29.0)*100))
    #For the case of the two firs events are not considered
    print("Percentuale prese tolte le prime 2:" +str((NTrueAlarm/(29.0-2))*100))
    
    return shapes2   

In [9]:
#ANOMLAIES CALCULATION
#basic ATTEMPT
model_params=None
import model_params
par=model_params.MODEL_PARAMS
par["modelParams"]["sensorParams"]["encoders"]={
                u'timestamp_timeOfDay': {
                        'fieldname': u'timestamp',
                        'name': u'timestamp_timeOfDay',
                        'timeOfDay': (21, 2),#21,9.5
                        'type': 'DateEncoder'
                },
                u'timestamp_dayOfWeek': None,
                u'timestamp_weekend': None,
                u'valuee':    {
                    'clipInput': True,
                    'fieldname': u'valuee',
                    'maxval': 65,
                    'minval': 0,
                    'n': 100, #500
                    'name': u'valuee',
                    'type': 'ScalarEncoder',
                    'w': 21
                }
            }
allData={}
anomalyLikelihood = anomaly_likelihood.AnomalyLikelihood(learningPeriod=288, estimationSamples=100, 
                historicWindowSize=8640)#600

##################################################
data={}
data["realData"]=in__.iloc[:,3:4].values
data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"],m=runHTM(par,
                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood)
allData["*************IN*************"]=data;
##################################################
#data={}
#data["realData"]=out__.iloc[:,3:4].values
#data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"]=runHTM(model_params.MODEL_PARAMS,
#                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood1)
#allData["*************OUT*************"]=data;
##################################################
#data={}
#data["realData"]=out__.iloc[:,3:4].values+in__.iloc[:,3:4].values
#data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"]=runHTM(model_params.MODEL_PARAMS,
#                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood2)
#allData["*************PLUS*************"]=data;
##################################################
#data={}
#data["realData"]=out__.iloc[:,3:4].values-in__.iloc[:,3:4].values
#data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"]=runHTM(model_params.MODEL_PARAMS,
#                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood)
#allData["*************MINUS*************"]=data;
##################################################

In [49]:
#PLOT ORIGINAL DATA
dataOriginal={}
dataOriginal["timestamps"]=data["timestamps"]
dataOriginal["ingressi"]=in__.iloc[:,3:4].values
plotData(dataOriginal, shapes, message="")

dataOriginal={}
dataOriginal["timestamps"]=data["timestamps"]
dataOriginal["uscite"]=out__.iloc[:,3:4].values
plotData(dataOriginal, shapes, message="Uscite")

#EVALUATION RESULTS, differents threshold for different total anomalies detected
thresholdAnomaly=0.081
for key, data in allData.items():
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    #plotData(data, shapes, message=key, shapes2=shapes2)
    
    thresholdAnomaly=0.0766#104
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    
    thresholdAnomaly=0.08798#70
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    
    thresholdAnomaly=0.132#48
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)

Uscite


*************IN*************
Anomalie totali:88
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0
*************IN*************
Anomalie totali:104
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0
*************IN*************
Anomalie totali:70
Anomalie reali:26
Percentuale prese:89.6551724138
Percentuale prese tolte le prime 2:96.2962962963
*************IN*************
Anomalie totali:48
Anomalie reali:24
Percentuale prese:82.7586206897
Percentuale prese tolte le prime 2:88.8888888889


In [50]:
#ANOMLAIES CALCULATION
#ATTEMPT with differrent parameter, IN and OUT used.
allData_={}
par=model_params.MODEL_PARAMS
par["modelParams"]["sensorParams"]["encoders"]={
                u'timestamp_timeOfDay': {
                        'fieldname': u'timestamp',
                        'name': u'timestamp_timeOfDay',
                        'timeOfDay': (21, 4.5),#21,2
                        'type': 'DateEncoder'
                },
                u'timestamp_dayOfWeek':None,
                u'timestamp_weekend': None,
                u'valuee':    {
                    'clipInput': True,
                    'fieldname': u'valuee',
                    'maxval': 65,
                    'minval': 0,
                    'n': 100, #500
                    'name': u'valuee',
                    'type': 'ScalarEncoder',
                    'w': 21
                }
            }

#best result: threshold=0.08
#Anomalie totali:91
#Anomalie reali:27
#Percentuale prese:93.1034482759
#Percentuale prese tolte le prime 2:100.0

anomalyLikelihood_0 = anomaly_likelihood.AnomalyLikelihood(learningPeriod=288, estimationSamples=100, 
                historicWindowSize=4000)#600
anomalyLikelihood_1 = anomaly_likelihood.AnomalyLikelihood(learningPeriod=288, estimationSamples=100, 
                historicWindowSize=4000)#600
#default:historicWindowSize=8640 but here less data so useless
#historicWindowSize tried 4000 2000--> very similar result
#historicWindowSize tried 600 300--> similar as before but with more general selection

#Tryed average of likelyhood between in e out and product, each work good bu less than IN. In particular average 
#very good since percent=92%(in 60 total alarm) but very difficult for percent=100%(in 150 total alarm)

##################################################
data={}
data["realData"]=in__.iloc[:,3:4].values
data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"],m=runHTM(par,
                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood_0)
allData_["*************IN*************"]=data;
##################################################
data={}
data["realData"]=out__.iloc[:,3:4].values
data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"],m=runHTM(par,
                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood_1)
allData_["*************OUT*************"]=data;
##################################################

In [51]:
#ATTEMPT with average anomalyLikelihood between anomalyLikelihood IN in and OUT
data={}
data["realData"]=out__.iloc[:,3:4].values
data["timestamps"]=allData_["*************IN*************"]["timestamps"]
data["anomalyLikelihood"]=(np.array(allData_["*************IN*************"]["anomalyLikelihood"])
                           +np.array(allData_["*************OUT*************"]["anomalyLikelihood"]))/2
allData_["*************AVERAGE aL*************"]=data;

In [52]:
#EVALUATION RESULTS
thresholdAnomaly=0.07
for key, data in allData_.items():
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    #plotData(data, key, shapes, shapes2)
print("\n -----IN-----")    
#best
data=allData_["*************IN*************"]
thresholdAnomaly=0.081 #0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
#104
thresholdAnomaly=0.07656 #0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
#70
thresholdAnomaly=0.082636314334 #0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
#48
thresholdAnomaly=0.105 #0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
#plotData(data, shapes, message="", shapes2=shapes2)

print("\n -----AVERAGE-----")
data=allData_["*************AVERAGE aL*************"]
#104
thresholdAnomaly=0.07147 #0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
#70
thresholdAnomaly=0.0765 #0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
#48
thresholdAnomaly=0.103#0.08
shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)

*************AVERAGE aL*************
Anomalie totali:127
Anomalie reali:26
Percentuale prese:89.6551724138
Percentuale prese tolte le prime 2:96.2962962963
*************OUT*************
Anomalie totali:140
Anomalie reali:20
Percentuale prese:68.9655172414
Percentuale prese tolte le prime 2:74.0740740741
*************IN*************
Anomalie totali:145
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0

 -----IN-----
Anomalie totali:80
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0
Anomalie totali:104
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0
Anomalie totali:71
Anomalie reali:26
Percentuale prese:89.6551724138
Percentuale prese tolte le prime 2:96.2962962963
Anomalie totali:48
Anomalie reali:24
Percentuale prese:82.7586206897
Percentuale prese tolte le prime 2:88.8888888889




 -----AVERAGE-----
Anomalie totali:104
Anomalie reali:26
Percentuale prese:89.6551724138
Percentuale prese tolte le prime 2:96.2962962963
Anomalie totali:70
Anomalie reali:25
Percentuale prese:86.2068965517
Percentuale prese tolte le prime 2:92.5925925926
Anomalie totali:48
Anomalie reali:23
Percentuale prese:79.3103448276
Percentuale prese tolte le prime 2:85.1851851852


In [27]:
##ATTEMPT with using the first 388 elements
#first combination of params
par=model_params.MODEL_PARAMS
par["modelParams"]["sensorParams"]["encoders"]={
                u'timestamp_timeOfDay': {
                        'fieldname': u'timestamp',
                        'name': u'timestamp_timeOfDay',
                        'timeOfDay': (21, 2),#21,9.5
                        'type': 'DateEncoder'
                },
                u'timestamp_dayOfWeek': None,
                u'timestamp_weekend': None,
                u'valuee':    {
                    'clipInput': True,
                    'fieldname': u'valuee',
                    'maxval': 65,
                    'minval': 0,
                    'n': 100, #500
                    'name': u'valuee',
                    'type': 'ScalarEncoder',
                    'w': 21
                }
            }

allData__={}

anomalyLikelihood__0 = anomaly_likelihood.AnomalyLikelihood(learningPeriod=288, estimationSamples=100, 
                historicWindowSize=4000)#600

##################################################
data={}
data["realData"]=in__.iloc[:,3:4].values 
#data["realData"]=np.concatenate((in__.iloc[:,3:4].values, in__.iloc[:,3:4].values), axis=0)
data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"], model=runHTM(par,
                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood__0)
inpt=in__.iloc[0:388,3:4].values 
ts_, as_, al_,m=runHTM(par, inpt, in__.iloc[0:388,1:3], anomalyLikelihood__0, model)
data["anomalyScore"][0:388]=as_
data["anomalyLikelihood"][0:388]=al_
allData__["*************IN*************"]=data;
##################################################

In [28]:
#EVALUATION RESULTS
for key, data in allData__.items():
    #104
    thresholdAnomaly=0.0774 #0.08
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    
    #70
    thresholdAnomaly=0.09 #0.08
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    
    #48    
    thresholdAnomaly=0.121 #0.08
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)

*************IN*************
Anomalie totali:104
Anomalie reali:28
Percentuale prese:96.5517241379
Percentuale prese tolte le prime 2:103.703703704
*************IN*************
Anomalie totali:70
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0
*************IN*************
Anomalie totali:48
Anomalie reali:26
Percentuale prese:89.6551724138
Percentuale prese tolte le prime 2:96.2962962963


In [29]:
##ATTEMPT with using the first 388 elements
#second combination of params
par=model_params.MODEL_PARAMS
par["modelParams"]["sensorParams"]["encoders"]={
                u'timestamp_timeOfDay': {
                        'fieldname': u'timestamp',
                        'name': u'timestamp_timeOfDay',
                        'timeOfDay': (21, 4.5),#21,2
                        'type': 'DateEncoder'
                },
                u'timestamp_dayOfWeek':None,
                u'timestamp_weekend': None,
                u'valuee':    {
                    'clipInput': True,
                    'fieldname': u'valuee',
                    'maxval': 65,
                    'minval': 0,
                    'n': 100, #500
                    'name': u'valuee',
                    'type': 'ScalarEncoder',
                    'w': 21
                }
            }
allData__2={}

anomalyLikelihood__0 = anomaly_likelihood.AnomalyLikelihood(learningPeriod=288, estimationSamples=100, 
                historicWindowSize=4000)#600

##################################################
data={}
data["realData"]=in__.iloc[:,3:4].values 
#data["realData"]=np.concatenate((in__.iloc[:,3:4].values, in__.iloc[:,3:4].values), axis=0)
data["timestamps"], data["anomalyScore"], data["anomalyLikelihood"], model=runHTM(par,
                                            data["realData"], in__.iloc[:,1:3], anomalyLikelihood__0)
inpt=in__.iloc[0:388,3:4].values 
ts_, as_, al_,m=runHTM(par, inpt, in__.iloc[0:388,1:3], anomalyLikelihood__0, model)
data["anomalyScore"][0:388]=as_
data["anomalyLikelihood"][0:388]=al_
allData__2["*************IN*************"]=data;
##################################################

In [30]:
#EVALUATION RESULTS
for key, data in allData__2.items():
    print(key)
    #best
    thresholdAnomaly=0.0813 #0.08
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    #plotData(data, key, shapes, shapes2)
    
    #104
    thresholdAnomaly=0.077 #0.08
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    
    #70
    thresholdAnomaly=0.0829 #0.08
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)
    
    #48    
    thresholdAnomaly=0.105535#0.08
    print(key)
    shapes2=evaluateResult(thresholdAnomaly, data["anomalyLikelihood"], data["timestamps"], frameEV)

*************IN*************
Anomalie totali:85
Anomalie reali:29
Percentuale prese:100.0
Percentuale prese tolte le prime 2:107.407407407
*************IN*************
Anomalie totali:104
Anomalie reali:29
Percentuale prese:100.0
Percentuale prese tolte le prime 2:107.407407407
*************IN*************
Anomalie totali:70
Anomalie reali:27
Percentuale prese:93.1034482759
Percentuale prese tolte le prime 2:100.0
*************IN*************
Anomalie totali:48
Anomalie reali:25
Percentuale prese:86.2068965517
Percentuale prese tolte le prime 2:92.5925925926
